In [19]:
import cv2
import mediapipe as mp
import os

In [20]:
nombre='sentir'  # Ir cambiando para crear las diferentes carpetas
direccion= './reconocimiento_imagenes/DATA/validacion'
carpeta= direccion + '/'+ nombre
if not os.path.exists(carpeta):
    print('Carpeta creada :' , carpeta)
    os.makedirs(carpeta)


Carpeta creada : ./reconocimiento_imagenes/DATA/validacion/sentir


In [21]:
cont=0   # contador para el nombre de las fotos

In [22]:
cap = cv2.VideoCapture(0)  # elegir camara web

 CREAMOS UN OBJETO QUE ALMACENA LA DETECCION Y SEGUIMIENTO DE LAS MANOS 

In [23]:
clase_manos = mp.solutions.hands
manos = clase_manos.Hands()

  METODO PARA DIBUJAR LAS MANOS

In [24]:
dibujo=mp.solutions.drawing_utils  # este metodo dibuja 21 puntos de la mano


while(1):
    ret,frame=cap.read()
    color = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    copia = frame.copy()
    resultado = manos.process(color)
    posiciones = []     # lista donde almacenamos coordenadas de los puntos
    #print(resultado.multi_hand_landmarks)    # si queremos ver si existe la deteccion
    
    if resultado.multi_hand_landmarks:
        for mano in resultado.multi_hand_landmarks:
            for id,lm in enumerate(mano.landmark):
                #print(id,lm)
                alto,ancho,c = frame.shape
                corx, cory =int(lm.x*ancho), int(lm.y*alto)
                posiciones.append([id ,corx,cory])
                
                dibujo.draw_landmarks(frame, mano, clase_manos.HAND_CONNECTIONS)#lineas entre puntos de la mano
            if len(posiciones)!= 0:
                pto_i1= posiciones [4]  # numeros de los puntos de la mano a detectar.
                pto_i2= posiciones [20]
                pto_i3= posiciones [12]
                pto_i4= posiciones [0]
                pto_i5= posiciones [9]  #punto central 
                
                x1,y1= (pto_i5[1]-200),(pto_i5[2]-200) # -80,-80 /+80,+80 definir cuadro de deteccion
                ancho, alto = (x1+200),(y1+200)        
                x2,y2= x1+ancho, y1+alto
                dedos_reg= copia[y1:y2, x1:x2] # copia las coordenas de arriba porque el recuadro es de 160 no de 80
                
                cv2.rectangle(frame, (x1,y1) , (x2,y2) , (0,255,0), 3)  # hace rectangulo
                
            dedos_reg=cv2.resize(dedos_reg,(200,200),interpolation=cv2.INTER_CUBIC) #redimensionamos las fotos
            
            cv2.imwrite(carpeta + "/Dedos_{}.jpg" .format(cont), dedos_reg)
            cont=cont+1
                
                
    #contador hasta 300 fotos   
    
    cv2.imshow("Video", frame)   
    k= cv2.waitKey(1)        
    if k==27 or cont >= 300:
        break
        
        
cap.release()
cv2.destroyAllWindows()
                

# CLASIFICADOR

In [2]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense,Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop

In [9]:
K.clear_session()   # para limpiar todo

In [26]:
datos_entrenamiento= './reconocimiento_imagenes/DATA/entrenamiento' # directorios donde estan las fotos
datos_validacion= './reconocimiento_imagenes/DATA/validacion' 

PARAMETROS 

In [11]:
iteraciones = 20  #numero de iteraciones para ajustar nuestro modelo
altura, longitud= 200,200  #tamaño imagenes de entrenamiento
batch_size = 1   #numero de imagenes que vamos a enviar
pasos=300/1      #numero de veces que se va a procesar la informacion en cada iteracion
pasos_validacion=300/1  #despues de cada iteracion validamos la anterior

filtrosconv1=32  
filtrosconv2=64  #   numero de filtros que aplicamos en cada iteracion     
filtrosconv3=128  # ponemos tres filtros para que abarque mejor los movimientos de la mano

tam_filtro1=(4,4)     # 4x4 ,3x3, 2x2
tam_filtro2=(3,3)  
tam_filtro3=(2,2)   #     tamaño del filtros (1,2,y 3)
  

tam_pool=(2,2)    #extrae caracteristicas de laimagen de 2x2 para tomar informacion muy detallada
clases= 5         # numero de palabras que vamos a hacer
lr= 0.0005        #ajustes de la red neuronal, para acceder a una solucion optima


PREPROCESAMIENTO DE LAS IMAGENES 

In [12]:
preprocesamiento_entre= ImageDataGenerator(
    rescale= 1./255, # pasar los pixeles de 0 255 | 0 a 1
    shear_range=0.3,  #generar nuestras imagenes inclinadas para un mejor entrenamiento
    zoom_range=0.3,  #generar imagenes con zoom para un mejor entrenamiento
    horizontal_flip= True) #invierte las imagenes para ajustar entrenamiento

In [13]:
preprocesamiento_val1=ImageDataGenerator( rescale = 1./255)

In [14]:
#IMAGEN ENTRENO
imagen_entreno=preprocesamiento_entre.flow_from_directory(
    datos_entrenamiento,  #va a tomar las fotos que ya almacenamos
    target_size= ( altura,longitud),
    batch_size = batch_size,
    class_mode = 'categorical') #clasificacion categorica = por clases


Found 323 images belonging to 2 classes.


In [15]:
#IMAGEN VALIDACION ( igual que arriba pero validacion)
imagen_validacion=preprocesamiento_entre.flow_from_directory(
    datos_entrenamiento,
    target_size= ( altura,longitud),
    batch_size= batch_size,
    class_mode = 'categorical')


Found 323 images belonging to 2 classes.


# CREAMOS LA RED NEURONAL CONVOLUCIONAL (CNN)

In [16]:
cnn= Sequential()  # red neuronal sequencial

Agregamos filtros con el fin de volver nuestra imagen muy profunda pero
pequeña.

In [18]:
#es una convolucion y realizamos config

cnn.add(Convolution2D(filtrosconv1,
                      tam_filtro1, padding='same', input_shape=(altura,longitud,3), activation='relu'))

cnn.add(MaxPooling2D(pool_size=tam_pool)) # despues de la primera capa vamos a tener una capa de max pooling y asignamos el tamaño
                                          # maxpooling es la extraccion de caracteristicas

    
cnn.add(Convolution2D(filtrosconv2,
                      tam_filtro2, padding='same', activation='relu')) # agregamos nueva capa

cnn.add(MaxPooling2D(pool_size=tam_pool))

#nueva capa

cnn.add(Convolution2D(filtrosconv3,
                      tam_filtro3, padding='same', activation='relu')) # agregamos nueva capa
cnn.add(MaxPooling2D(pool_size=tam_pool))

Convertir imagen profunda en imagen plana, para tener una dimension 

In [19]:
cnn.add(Flatten())  # aplanamos la imagen
cnn.add(Dense(640,activation='relu'))  # asignamos 640 neuronas ( 256 ,2 clases, 640,5 clases para calcular con regla de 3)
cnn.add(Dropout(0.5)) # apagamos el 50% de las neuronas para ajustar la red( solo entranamiento e validacion utilizar todas)
cnn.add(Dense(clases,activation='softmax'))   # dice la probabilidad de que sea de una clase o de otra 


Agregamos parametros para optimizar el modelo

Durante el entrenamiento, tiene autoevaluacion, se optimiza con Adam y la metrica Accuracy

In [36]:
optimizar=tensorflow.keras.optimizers.Adam(learning_rate=lr)


In [39]:

cnn.compile(loss='categorical_crossentropy', optimizer=optimizar, metrics=['accuracy'])

ValueError: Could not interpret optimizer identifier: <keras.optimizer_v2.adam.Adam object at 0x0000013AE4BC9F40>

Entrenamos la red

In [ ]:
cnn.fit(Imagen_entreno , steps_per_epoch = pasos, epochs = iteraciones , validation_date=imagen_validacion , validacion_steps=pasos_validacion)


Guardamos el modelo

In [ ]:
cnn.save('modelopalabras.h5')
cnn.save_weights('pesospalabras.h5')

# Prediccion 

In [51]:
import cv2
import mediapipe as mp
import os
import numpy as np
from keras_preprocessing.image import load_img ,img_to_array
from keras.models import load_model

In [ ]:
modelo= './modelopalabras.h5'
peso='./pesospalabras.h5'
cnn=load_model(modelo)   # cargar modelo
cnn.load_weights(peso)   #cargar peso

direccion='./reconocimiento_imagenes/DATA/validacion'
dire_ing = os.listdir(direccion)
print('Nombres:  ', dire_ing)

In [ ]:
cap=cv2.VideoCapture(0)   # leemos camara

Creamos un objeto que va a almacenar la deteccion y el seguimiento de las manos

In [ ]:
clase_manos=mp.solutions.hands
manos=clase_manos.Hands()  # especificaciones es word

dibujo=mp.solutions.drawing_utils


while(1):
    ret,frame=cap.read()
    color = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    copia = frame.copy()
    resultado = manos.process(color)
    posiciones = []     # lista donde almacenamos coordenadas de los puntos
    #print(resultado.multi_hand_landmarks)    # si queremos ver si existe la deteccion
    
    if resultado.multi_hand_landmarks:
        for mano in resultado.multi_hand_landmarks:
            for id,lm in enumerate(mano.landmark):
                #print(id,lm)
                alto,ancho,c = frame.shape
                corx, cory =int(lm.x*ancho), int(lm.y*alto)
                posiciones.append([id ,corx,cory])
                
                dibujo.draw_landmarks(frame, mano, clase_manos.HAND_CONNECTIONS)#lineas entre puntos de la mano
            if len(posiciones)!= 0:
                pto_i1= posiciones [4]  # numeros de los puntos de la mano a detectar.
                pto_i2= posiciones [20]
                pto_i3= posiciones [12]
                pto_i4= posiciones [0]
                pto_i5= posiciones [9]  #punto central 
                
                x1,y1= (pto_i5[1]-200),(pto_i5[2]-200) # -80,-80 /+80,+80 definir cuadro de deteccion
                ancho, alto = (x1+200),(y1+200)        
                x2,y2= x1+ancho, y1+alto
                dedos_reg= copia[y1:y2, x1:x2] # copia las coordenas de arriba porque el recuadro es de 160 no de 80
                dedos_reg=cv2.resize(dedos_reg,(200,200),interpolation=cv2.INTER_CUBIC) #redimensionamos las fotos
                x= img_to_array(dedos_reg)
                x=np.expand.dims(x, axis=0)
                vector=cnn.predict(x)
                resultado=vector[0] = [1,0] | [0,1]
                respuesta=np.argmax(resultado)
                
                #vamos poniendo condiciones dependienedo del numero de carpetas de imagenes que tenemos
                if respuesta==0:
                    print(resultado)
                    cv2.rectangle(frame, (x1,y1), (x2,y2) , (0,255,0),3)
                    cv2.putText(frame, '{}'.format(dire_img[0]),(x1 , y1-5), 1, 1.3, (0, 255, 0), 1 ,cv2.LINE_AA )
                elif respuesta==1:
                    print(resultado)
                    cv2.rectangle(frame, (x1,y1), (x2,y2) , (0,0,255),3)
                    cv2.putText(frame, '{}'.format(dire_img[1]),(x1 , y1-5), 1, 1.3, (0, 255, 0), 1 ,cv2.LINE_AA )
                elif respuesta==2:
                    print(resultado)
                    cv2.rectangle(frame, (x1,y1), (x2,y2) , (255,0,0),3)
                    cv2.putText(frame, '{}'.format (dire_img[2]),(x1 , y1-5), 1, 1.3, (0, 255, 0), 1 ,cv2.LINE_AA )
                elif respuesta==3:
                    print(resultado)
                    cv2.rectangle(frame, (x1,y1), (x2,y2) , (255,0,255),3)
                    cv2.putText(frame, '{}'.format(dire_img[3]),(x1 , y1-5), 1, 1.3, (0, 255, 0), 1 ,cv2.LINE_AA )
                elif respuesta==4:
                    print(resultado)
                    cv2.rectangle(frame, (x1,y1), (x2,y2) , (0,255,255),3)
                    cv2.putText(frame, '{}'.format(dire_img[4]),(x1 , y1-5), 1, 1.3, (0, 255, 0), 1 ,cv2.LINE_AA )
                else:
                    cv2.putText(frame, ' PALABRA DESCONOCIDA ', (x1 , y1 - 5),1, 1.3, (0, 255, 255), 1 ,cv2.LINE_AA )
                    
    cv2.imshow('Video',frame)
    k= cv2.waikey(1)
    if k ==27:
        break
        
cap.release()
cv2.destroyAllWindows()


